In [1]:
from Library import *
from Data_Preparation import *
#from Data_Preparation2 import *
import folium 
import scipy
import sklearn
from IPython.display import display
from sklearn import preprocessing
from itertools import combinations
import pickle
from sklearn import decomposition
from icecream import ic 
import os
from tqdm import tqdm
import requests
import nbimporter
from datetime import datetime
from sklearn.mixture import GaussianMixture
from sklearn.metrics import roc_curve, auc , roc_auc_score, r2_score, accuracy_score , silhouette_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from datetime import datetime, time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.cluster import DBSCAN
from IPython.display import Markdown, display
import geopandas as gpd
from shapely.geometry import Point
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [2]:
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings('ignore')

In [3]:


def get_neighborhood_name(latitude, longitude):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}"
    
    try:
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code != 200:
#             print( f"Error: Received status code {response.status_code}")
            return get_neighborhood_name(latitude, longitude)

        
        # Parse the response JSON
        data = response.json()
        
        if 'address' in data:
            if 'neighbourhood' in data['address']:
                return data['address']['neighbourhood']
            elif 'suburb' in data['address']:
                return data['address']['suburb']
            elif 'town' in data['address']:
                return data['address']['town']
            elif 'city' in data['address']:
                return data['address']['city']
            else:
                return "Neighborhood not found"
        else:
            return "Neighborhood not found"
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    
    except ValueError as e:
        print(f"JSON decoding failed: {e}")


# Example usage:


latitude = 32.108205  # Example latitude
longitude = 34.823596  # Example longitude

neighborhood_name = get_neighborhood_name(latitude, longitude)
print("Neighborhood:", neighborhood_name)


Neighborhood: הדר-יוסף


In [1]:
import requests

def translate_hebrew_to_english(text):
    url = "https://libretranslate.com/translate"
    
    # Setting up the payload with source language (Hebrew) and target language (English)
    payload = {
        "q": text,
        "source": "he",  # Hebrew
        "target": "en",  # English
        "format": "text"
    }
    
    # Making the request to LibreTranslate
    response = requests.post(url, data=payload)
    
    # Check if the request was successful
    if response.status_code == 200:
        translated_text = response.json().get("translatedText")
        return translated_text
    else:
        return "Error: Unable to translate."

# Example usage
hebrew_text = "שלום עולם"
english_translation = translate_hebrew_to_english(hebrew_text)
print(english_translation)


Error: Unable to translate.


In [5]:

df = concated_data_with_feat(460631)
df.columns = df.columns.str.upper()
df["START_HOUR"] = pd.to_datetime(df["START_DRIVE"]).dt.hour
df["END_HOUR"] = pd.to_datetime(df["END_DRIVE"]).dt.hour
df["DATE"] = pd.to_datetime(df["START_DRIVE"]).dt.date
df["DAY_OF_WEEK"] = df["DATE"].apply(lambda x: x.strftime("%A"))
df.index = df['DRIVE_ID']
df = df.drop(["START_DRIVE", "END_DRIVE"], axis=1)

In [6]:
columns_og  = ['min_speed', 'length_edge_v', '90_max_angular_acc', 'length_edge_h','90_50%_angular_acc', '180_min_speed', '50_min_speed','50%_angular_acc', '90_std_acceleration_est_1', 'trip_length_km','mean_speed_diff', '90_75%_angular_acc', '70_min_angular_acc','90_mean_speed_diff', '90_50%_acceleration_est_1']

columns_og = [x.upper() for x in columns_og]
columns = columns_og +['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']

In [7]:
df.columns = df.columns.str.lower()

scaled_df = scaling_by_coors_feat(df,460631)
df.columns = df.columns.str.upper()
scaled_df.index = scaled_df['DRIVE_ID']
scaled_df

,DRIVE_ID,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,MEAN_SPEED,STD_SPEED,MIN_SPEED,25%_SPEED,50%_SPEED,...,SPEED1,SPEED2,SPEED3,SUM_EVENTS,VEHICLE_ID,START_HOUR,END_HOUR,DATE,DAY_OF_WEEK,CORRS
DRIVE_ID,,,,,,,,,,,,,,,,,,,,,
12808103918,12808103918,32.165485,34.857773,32.112946,34.818150,0.821431,0.806878,-0.278291,-0.188115,0.812124,...,0,0,0,0,460631.0,6,6,2018-01-05,Friday,"(32.17, 34.86, 32.11, 34.82)"
12809856354,12809856354,32.070026,34.829456,32.167912,34.799416,2.205333,1.770897,0.000000,1.373324,2.377689,...,0,0,0,0,460631.0,12,13,2018-01-05,Friday,"(32.07, 34.83, 32.17, 34.8)"
12810378837,12810378837,32.167870,34.799389,32.165478,34.857697,0.329747,1.128123,-0.422116,-1.026391,0.640607,...,0,0,0,0,460631.0,14,15,2018-01-05,Friday,"(32.17, 34.8, 32.17, 34.86)"
12812256953,12812256953,32.165455,34.857704,32.160561,34.807297,-1.235424,0.121860,-0.507275,-2.141184,-2.003283,...,0,0,0,0,460631.0,11,11,2018-01-06,Saturday,"(32.17, 34.86, 32.16, 34.81)"
12812532671,12812532671,32.155720,34.795891,32.165466,34.857758,-0.126392,0.295826,-0.447265,-0.450698,0.199608,...,0,0,0,0,460631.0,14,14,2018-01-06,Saturday,"(32.16, 34.8, 32.17, 34.86)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52964046721,52964046721,32.165543,34.857712,32.187866,34.877132,-1.630534,-1.516066,-0.277528,-0.583630,-1.220824,...,0,0,0,1,460631.0,8,9,2019-12-31,Tuesday,"(32.17, 34.86, 32.19, 34.88)"
52965009685,52965009685,32.187855,34.877079,32.112881,34.818039,1.502280,1.145142,-0.301797,0.714434,2.958874,...,0,0,0,0,460631.0,11,11,2019-12-31,Tuesday,"(32.19, 34.88, 32.11, 34.82)"
52967424318,52967424318,32.112736,34.818058,32.165535,34.857666,-0.349912,-0.162569,0.618543,-0.459422,-0.377336,...,0,0,0,0,460631.0,17,17,2019-12-31,Tuesday,"(32.11, 34.82, 32.17, 34.86)"


In [8]:
cols_to_exclude = ['CORRS','DAY_OF_WEEK','DATE','VEHICLE_ID','DRIVE_ID','INDEX','START_LATITUDE', 'START_LONGITUDE', 'END_LATITUDE', 'END_LONGITUDE','END_HOUR','START_HOUR']

mask = scaled_df.drop(cols_to_exclude,axis=1).apply(lambda x: (x > 6) | (x < -6)).any(axis=1)
cleaned = scaled_df[~mask]
cleaned = cleaned[(cleaned['START_LATITUDE'] != cleaned['END_LATITUDE'])&(cleaned['START_LONGITUDE'] != cleaned['END_LONGITUDE'])]
cleaned

,DRIVE_ID,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,MEAN_SPEED,STD_SPEED,MIN_SPEED,25%_SPEED,50%_SPEED,...,SPEED1,SPEED2,SPEED3,SUM_EVENTS,VEHICLE_ID,START_HOUR,END_HOUR,DATE,DAY_OF_WEEK,CORRS
DRIVE_ID,,,,,,,,,,,,,,,,,,,,,
12808103918,12808103918,32.165485,34.857773,32.112946,34.818150,0.821431,0.806878,-0.278291,-0.188115,0.812124,...,0,0,0,0,460631.0,6,6,2018-01-05,Friday,"(32.17, 34.86, 32.11, 34.82)"
12809856354,12809856354,32.070026,34.829456,32.167912,34.799416,2.205333,1.770897,0.000000,1.373324,2.377689,...,0,0,0,0,460631.0,12,13,2018-01-05,Friday,"(32.07, 34.83, 32.17, 34.8)"
12810378837,12810378837,32.167870,34.799389,32.165478,34.857697,0.329747,1.128123,-0.422116,-1.026391,0.640607,...,0,0,0,0,460631.0,14,15,2018-01-05,Friday,"(32.17, 34.8, 32.17, 34.86)"
12812256953,12812256953,32.165455,34.857704,32.160561,34.807297,-1.235424,0.121860,-0.507275,-2.141184,-2.003283,...,0,0,0,0,460631.0,11,11,2018-01-06,Saturday,"(32.17, 34.86, 32.16, 34.81)"
12812755403,12812755403,32.165455,34.857773,32.062527,34.769505,0.462054,0.125528,0.000000,1.025099,0.502478,...,0,0,0,2,460631.0,16,17,2018-01-06,Saturday,"(32.17, 34.86, 32.06, 34.77)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52951703968,52951703968,32.115570,34.801773,32.165459,34.857712,0.282161,-0.250425,0.732331,0.394271,0.498423,...,0,0,0,0,460631.0,19,19,2019-12-29,Sunday,"(32.12, 34.8, 32.17, 34.86)"
52965009685,52965009685,32.187855,34.877079,32.112881,34.818039,1.502280,1.145142,-0.301797,0.714434,2.958874,...,0,0,0,0,460631.0,11,11,2019-12-31,Tuesday,"(32.19, 34.88, 32.11, 34.82)"
52967424318,52967424318,32.112736,34.818058,32.165535,34.857666,-0.349912,-0.162569,0.618543,-0.459422,-0.377336,...,0,0,0,0,460631.0,17,17,2019-12-31,Tuesday,"(32.11, 34.82, 32.17, 34.86)"


In [61]:
# cleaned['NEIGHBORHOOD'] = cleaned.apply(lambda row: get_neighborhood_name(round(row['END_LATITUDE'],4), round(row['END_LONGITUDE'],4)), axis=1)

from tqdm import tqdm

# Initialize the NEIGHBORHOOD column with None or empty values
cleaned['end_NEIGHBORHOOD'] = None
cleaned['start_NEIGHBORHOOD'] = None
# Iterate over rows with tqdm progress bar
for idx, row in tqdm(cleaned.iterrows(), total=len(cleaned)):
    cleaned.at[idx, 'end_NEIGHBORHOOD'] = get_neighborhood_name(
        round(row['END_LATITUDE'], 4), 
        round(row['END_LONGITUDE'], 4)
    )
    cleaned.at[idx, 'start_NEIGHBORHOOD'] = get_neighborhood_name(
        round(row['START_LATITUDE'], 4), 
        round(row['START_LONGITUDE'], 4)
    )
    
df_for_decision_tree = cleaned[['end_NEIGHBORHOOD','start_NEIGHBORHOOD','NEIGHBORHOOD','DAY_OF_WEEK','DATE','VEHICLE_ID','DRIVE_ID','INDEX','START_LATITUDE', 'START_LONGITUDE', 'END_LATITUDE', 'END_LONGITUDE','END_HOUR','START_HOUR']]

100%|███████████████████████████████████████| 1867/1867 [2:14:19<00:00,  4.32s/it]


In [68]:
df_for_decision_tree = cleaned[['end_NEIGHBORHOOD','start_NEIGHBORHOOD','DAY_OF_WEEK','START_LATITUDE', 'START_LONGITUDE', 'END_LATITUDE', 'END_LONGITUDE','END_HOUR','START_HOUR']]

In [69]:
df_for_decision_tree

,end_NEIGHBORHOOD,start_NEIGHBORHOOD,DAY_OF_WEEK,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,END_HOUR,START_HOUR
DRIVE_ID,,,,,,,,,
12808103918,הדר-יוסף,נוה עמל,Friday,32.165485,34.857773,32.112946,34.818150,6,6
12809856354,הרצליה פיתוח,רמת עמידר,Friday,32.070026,34.829456,32.167912,34.799416,13,12
12810378837,נוה עמל,הרצליה פיתוח,Friday,32.167870,34.799389,32.165478,34.857697,15,14
12812256953,הרצליה ב',נוה עמל,Saturday,32.165455,34.857704,32.160561,34.807297,11,11
12812755403,לב תל-אביב,נוה עמל,Saturday,32.165455,34.857773,32.062527,34.769505,17,16
...,...,...,...,...,...,...,...,...,...
52951703968,נוה עמל,"אוניברסיטת ת""א",Sunday,32.115570,34.801773,32.165459,34.857712,19,19
52965009685,הדר-יוסף,Shazar,Tuesday,32.187855,34.877079,32.112881,34.818039,11,11
52967424318,נוה עמל,הדר-יוסף,Tuesday,32.112736,34.818058,32.165535,34.857666,17,17


In [85]:
import os

# Your DataFrame 'df'
file_name = "460631.csv"  # File name
save_path = "/bigdata/users-home/dor/transpotation research/agg_data/pattern_df/"

# Ensure the directory exists
# os.makedirs(save_path, exist_ok=True)

# Full path to save the file
full_path = os.path.join(save_path, file_name)

# Save the DataFrame as CSV
df_for_decision_tree.to_csv(full_path, index=False)

print(f"DataFrame saved to {full_path}")


DataFrame saved to /bigdata/users-home/dor/transpotation research/agg_data/pattern_df/460631.csv


In [9]:
# Define the path to the CSV file
file_name = "460631.csv"
file_path = "/bigdata/users-home/dor/transpotation research/agg_data/pattern_df/"  # Path to your file

# Full path to the CSV file
full_path = os.path.join(file_path, file_name)

# Load the CSV file into a DataFrame
df_for_decision_tree = pd.read_csv(full_path)

In [10]:
grouped_df = cleaned.copy()
grouped_df["START_LATITUDE"] = scaled_df["START_LATITUDE"].round(3)
grouped_df["START_LONGITUDE"] = scaled_df["START_LONGITUDE"].round(3)
grouped_df["END_LATITUDE"] = scaled_df["END_LATITUDE"].round(3)
grouped_df["END_LONGITUDE"] = scaled_df["END_LONGITUDE"].round(3)
grouped_df = grouped_df.groupby([ 'END_LATITUDE', 'END_LONGITUDE']).size().reset_index(name='num_drives')


In [11]:
most_driven_cor = grouped_df.sort_values(by='num_drives', ascending=False).iloc[0]
# Extract latitude and longitude from the selected row
latitude = most_driven_cor['END_LATITUDE']
longitude = most_driven_cor['END_LONGITUDE']

# Create a tuple of the most driven coordinates
home_cor = (latitude, longitude)

agg_drives_filt = grouped_df[(grouped_df['num_drives'] > 7)&((grouped_df['END_LATITUDE']!=latitude) & (grouped_df['END_LONGITUDE']!=longitude))].sort_values(by=[ 'num_drives'], ascending=[ False])

# agg_drives_filt = agg_drives_filt[(agg_drives_filt['START_LATITUDE'] != agg_drives_filt['END_LATITUDE'])&(agg_drives_filt['START_LONGITUDE'] != agg_drives_filt['END_LONGITUDE'])]
agg_drives_filt['NEIGHBORHOOD'] = agg_drives_filt.apply(lambda row: translate_hebrew_to_english(get_neighborhood_name(row['END_LATITUDE'], row['END_LONGITUDE'])), axis=1)
agg_drives_filt

,END_LATITUDE,END_LONGITUDE,num_drives,NEIGHBORHOOD
67,32.113,34.818,140,Error: Unable to translate.
77,32.116,34.802,72,Error: Unable to translate.
278,32.188,34.846,57,Error: Unable to translate.
137,32.158,34.795,46,Error: Unable to translate.
57,32.110,34.807,35,Error: Unable to translate.
74,32.115,34.818,31,Error: Unable to translate.
269,32.185,34.853,26,Error: Unable to translate.
123,32.150,34.846,17,Error: Unable to translate.
75,32.115,34.819,14,Error: Unable to translate.
236,32.171,34.825,13,Error: Unable to translate.


In [12]:
def tag_drives(agg_drives,driver=1):
    drives_df = agg_drives.copy()
    drives_df['driver'] = 0
    driver_dict = {}
#     agg_drives.at[3263176, 'driver'] = 0
    for i, row in drives_df.iterrows():
#         agg_drives.at[i, 'driver'] = int(0)
        if  i == drives_df.index[0]:
            drives_df.at[i, 'driver'] = driver
            driver_dict[i] = row
        else:
            for j in driver_dict:
                if (
                    row['NEIGHBORHOOD'] == driver_dict[j]['NEIGHBORHOOD']):
#                     and row['END_LATITUDE'] == driver_dict[j]['END_LATITUDE']
#                     and row['END_LONGITUDE'] == driver_dict[j]['END_LONGITUDE'] ):

                        drives_df.at[i, 'driver'] = driver
                        break  # Exit the loop once a match is found
    return drives_df[drives_df['driver']==1] , drives_df[drives_df['driver']==0]

In [13]:
def get_drives(df, drives_group, with_returns=True):   
    # List to store indices of matching drives
    matching_drives = []

    # Iterate over each row in the dataframe and drives_group
    for index_df, row_df in df.iterrows():
        for index_dg, row_dg in drives_group.iterrows():
            # Check if the end coordinates of the current drive in df match
            # the end coordinates of the current drive in drives_group
            if (round(row_df['END_LATITUDE'], 3) == row_dg['END_LATITUDE'] and
                round(row_df['END_LONGITUDE'], 3) == row_dg['END_LONGITUDE']):
                # If match found, append index of the drive in df to matching_drives
                matching_drives.append(index_df)
                # Break the inner loop as we've found a match
                break

    if with_returns:
        # Iterate over the filtered dataframe (df) based on matching indices
        for index_df, row_df in df[df.index.isin(matching_drives)].iterrows():
            # Iterate again over the entire dataframe to find drives starting where
            # the matching drives end and on the same date
            for index, row in df.iterrows():
                if (round(row_df['END_LATITUDE'], 3) == round(row['START_LATITUDE'], 3) and
                    round(row_df['END_LONGITUDE'], 3) == round(row['START_LONGITUDE'], 3) and
                    row_df['DATE'] == row['DATE']):
                    # If conditions match, append index of the drive in df to following_drives
                    matching_drives.append(index)
                    break
    
    return matching_drives


## creating fixed drives clustrings with the return drive (based on END_LONGITUDE)

In [14]:
dict_drives = {}
dict_sum ={}
dict_cor = {}
agg_drives2 = agg_drives_filt.copy()
i=1
while len(agg_drives2)>1:
    tag_drives_group,agg_drives2 = tag_drives(agg_drives2)
    
    driver1_lst  = get_drives(df, tag_drives_group)
    driver1_df = cleaned[cleaned.index.isin(driver1_lst)][columns_og[:15]+['END_LATITUDE','END_LONGITUDE']]
    sum_drives = len(driver1_df)
    if sum_drives > 0:
        dict_cor[i] = (float(tag_drives_group.head(1)['END_LATITUDE']), float(tag_drives_group.head(1)['END_LONGITUDE']))

        dict_sum[i] = sum_drives
        dict_drives[i] = driver1_df
        
    i+=1
dict_sum = {k: v for k, v in sorted(dict_sum.items(), key=lambda item: item[1], reverse=True)}

dict_cor = {k: dict_cor[k] for k in dict_sum.keys()}
dict_drives = {k: dict_drives[k] for k in dict_sum.keys()}

print(dict_sum)


{1: 941}


In [15]:
keys_to_delete = [key for key, value in dict_sum.items() if value < 50]
for key in keys_to_delete:
    del dict_sum[key]
    del dict_drives[key]
    del dict_cor[key]

print(dict_sum)

{1: 941}


In [16]:
def remap_keys(dicts):
    # Find the minimum key in all dictionaries
    min_key = min(min(d) for d in dicts)
    # Create a mapping from old keys to new keys
    key_mapping = {old_key: idx + min_key for idx, old_key in enumerate(sorted(set.union(*map(set, dicts))))}
    # Remap keys in all dictionaries
    remapped_dicts = [{key_mapping[key]: value for key, value in d.items()} for d in dicts]
    return tuple(remapped_dicts)

In [17]:
lst = remap_keys([dict_sum, dict_drives, dict_cor])
dict_sum =lst[0]
dict_drives = lst[1]
dict_cor = lst[2]

In [18]:
dict_sum = {k: v for k, v in sorted(dict_sum.items(), key=lambda item: item[1], reverse=True)}

dict_cor = {k: dict_cor[k] for k in dict_sum.keys()}
dict_drives = {k: dict_drives[k] for k in dict_sum.keys()}

In [19]:
neigh_dict = {}
for group in dict_cor:
    neigh_dict[group] = get_neighborhood_name(dict_cor[group][0], dict_cor[group][1])
    print("Neighborhood:", neigh_dict[group],group)

Neighborhood: הדר-יוסף 1


In [72]:
# index_lst = []
# for key in dict_drives:
#     for i , row in dict_drives[key].iterrows():
#         index_lst.append(i)

In [73]:
# df_for_decision_tree = df_for_decision_tree[df_for_decision_tree.index.isin(index_lst)]
# df_for_decision_tree

,end_NEIGHBORHOOD,start_NEIGHBORHOOD,DAY_OF_WEEK,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,END_HOUR,START_HOUR
DRIVE_ID,,,,,,,,,
12808103918,הדר-יוסף,נוה עמל,Friday,32.165485,34.857773,32.112946,34.818150,6,6
12813279223,"אוניברסיטת ת""א",לב תל-אביב,Saturday,32.062527,34.769505,32.115765,34.801937,19,18
12813348009,נוה עמל,"אוניברסיטת ת""א",Saturday,32.115761,34.801910,32.165424,34.857723,19,19
12817574355,אפקה,ויצמן,Sunday,32.163670,34.832230,32.115990,34.802086,12,11
12820014729,נוה עמל,"אוניברסיטת ת""א",Sunday,32.116177,34.801956,32.165424,34.857735,19,19
...,...,...,...,...,...,...,...,...,...
52926274694,נוה עמל,הדר-יוסף,Wednesday,32.112808,34.817940,32.165520,34.857681,20,20
52930860048,נוה עמל,הדר-יוסף,Thursday,32.112965,34.818047,32.165527,34.857662,12,12
52951703968,נוה עמל,"אוניברסיטת ת""א",Sunday,32.115570,34.801773,32.165459,34.857712,19,19


In [41]:
def create_dummies(df, column_name):

    # Check if the column_name exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")
    
    # Create dummy variables
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    
    # Drop the original column and concatenate the dummies
    df = df.drop(column_name, axis=1)
    df = pd.concat([df, dummies], axis=1)
    
    return df
for column_name in ['end_NEIGHBORHOOD','start_NEIGHBORHOOD','DAY_OF_WEEK','END_HOUR','START_HOUR']:
    
    df_for_decision_tree = create_dummies(df_for_decision_tree, column_name)

In [20]:
df_for_decision_tree

,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,end_NEIGHBORHOOD_Shazar,"end_NEIGHBORHOOD_אוניברסיטת ת""א",end_NEIGHBORHOOD_איזור תעשייה רמת סיב,end_NEIGHBORHOOD_אפקה,end_NEIGHBORHOOD_גבעת הלבונה,end_NEIGHBORHOOD_גורדון,...,START_HOUR_14,START_HOUR_15,START_HOUR_16,START_HOUR_17,START_HOUR_18,START_HOUR_19,START_HOUR_20,START_HOUR_21,START_HOUR_22,label
0,32.165485,34.857773,32.112946,34.818150,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,32.062527,34.769505,32.115765,34.801937,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
2,32.115761,34.801910,32.165424,34.857723,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
3,32.163670,34.832230,32.115990,34.802086,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
4,32.116177,34.801956,32.165424,34.857735,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,32.112808,34.817940,32.165520,34.857681,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
771,32.112965,34.818047,32.165527,34.857662,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
772,32.115570,34.801773,32.165459,34.857712,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
773,32.187855,34.877079,32.112881,34.818039,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [79]:
# df_for_decision_tree['label'] = None
# for key in dict_drives:
#     for i , row in dict_drives[key].iterrows():
#         df_for_decision_tree.at[i, 'label'] = key
# df_for_decision_tree

,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,end_NEIGHBORHOOD_Shazar,"end_NEIGHBORHOOD_אוניברסיטת ת""א",end_NEIGHBORHOOD_איזור תעשייה רמת סיב,end_NEIGHBORHOOD_אפקה,end_NEIGHBORHOOD_גבעת הלבונה,end_NEIGHBORHOOD_גורדון,...,START_HOUR_14,START_HOUR_15,START_HOUR_16,START_HOUR_17,START_HOUR_18,START_HOUR_19,START_HOUR_20,START_HOUR_21,START_HOUR_22,label
DRIVE_ID,,,,,,,,,,,,,,,,,,,,,
12808103918,32.165485,34.857773,32.112946,34.818150,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12813279223,32.062527,34.769505,32.115765,34.801937,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
12813348009,32.115761,34.801910,32.165424,34.857723,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
12817574355,32.163670,34.832230,32.115990,34.802086,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
12820014729,32.116177,34.801956,32.165424,34.857735,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52926274694,32.112808,34.817940,32.165520,34.857681,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
52930860048,32.112965,34.818047,32.165527,34.857662,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52951703968,32.115570,34.801773,32.165459,34.857712,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2


In [33]:
import pandas as pd

def reverse_dummies(df, prefix):
    """
    Reverts dummy variables back to the original categorical values.
    
    Args:
        df (pd.DataFrame): The DataFrame containing dummy variables.
        prefix (str): The prefix used for dummy variable columns.
        
    Returns:
        pd.DataFrame: DataFrame with dummy variables reversed.
    """
    # Identify dummy columns
    dummy_columns = [col for col in df.columns if col.startswith(prefix)]
    
    if not dummy_columns:
        raise ValueError(f"No dummy columns found with prefix '{prefix}'")
    
    # Create a DataFrame with the dummy variables only
    dummy_df = df[dummy_columns]
    
    # Get the original column values by finding the index of the maximum value
    original_column = dummy_df.idxmax(axis=1)
    
    # Extract the category from the column names
    original_values = original_column.str.replace(prefix + '_', '')
    
    # Add the original column to the DataFrame and drop dummy columns
    df = df.drop(dummy_columns, axis=1)
    df[prefix] = original_values
    
    return df
# Assuming 'df_for_decision_tree' is your DataFrame and you want to reverse dummies for 'end_NEIGHBORHOOD'
df_for_decision_tree = reverse_dummies(df_for_decision_tree, 'end_NEIGHBORHOOD')

# Repeat for other columns if needed
df_for_decision_tree = reverse_dummies(df_for_decision_tree, 'start_NEIGHBORHOOD')
df_for_decision_tree = reverse_dummies(df_for_decision_tree, 'DAY_OF_WEEK')
df_for_decision_tree = reverse_dummies(df_for_decision_tree, 'END_HOUR')
df_for_decision_tree = reverse_dummies(df_for_decision_tree, 'START_HOUR')


In [35]:
from sklearn.preprocessing import LabelEncoder

def encode_nominal_columns(df):
    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()
    
    # Combine the end_NEIGHBORHOOD and start_NEIGHBORHOOD columns for consistent mapping
    combined_neighborhoods = df['end_NEIGHBORHOOD'].append(df['start_NEIGHBORHOOD']).unique()
    
    # Fit the label encoder on the combined neighborhoods
    label_encoder.fit(combined_neighborhoods)
    
    # Encode the start_NEIGHBORHOOD and end_NEIGHBORHOOD columns
    df['start_NEIGHBORHOOD_encoded'] = label_encoder.transform(df['start_NEIGHBORHOOD'])
    df['end_NEIGHBORHOOD_encoded'] = label_encoder.transform(df['end_NEIGHBORHOOD'])
    
    # Encode the DAY_OF_WEEK column
    df['DAY_OF_WEEK_encoded'] = label_encoder.fit_transform(df['DAY_OF_WEEK'])
    
    return df
df_for_decision_tree = encode_nominal_columns(df_for_decision_tree)

In [36]:
df_for_decision_tree

,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,label,end_NEIGHBORHOOD,start_NEIGHBORHOOD,DAY_OF_WEEK,END_HOUR,START_HOUR,start_NEIGHBORHOOD_encoded,end_NEIGHBORHOOD_encoded,DAY_OF_WEEK_encoded
0,32.165485,34.857773,32.112946,34.818150,1,הדר-יוסף,נוה עמל,Friday,6,6,27,11,0
1,32.062527,34.769505,32.115765,34.801937,2,"אוניברסיטת ת""א",לב תל-אביב,Saturday,19,18,22,2,2
2,32.115761,34.801910,32.165424,34.857723,2,נוה עמל,"אוניברסיטת ת""א",Saturday,19,19,2,27,2
3,32.163670,34.832230,32.115990,34.802086,2,אפקה,ויצמן,Sunday,12,11,18,4,3
4,32.116177,34.801956,32.165424,34.857735,2,נוה עמל,"אוניברסיטת ת""א",Sunday,19,19,2,27,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,32.112808,34.817940,32.165520,34.857681,1,נוה עמל,הדר-יוסף,Wednesday,20,20,11,27,6
771,32.112965,34.818047,32.165527,34.857662,1,נוה עמל,הדר-יוסף,Thursday,12,12,11,27,4
772,32.115570,34.801773,32.165459,34.857712,2,נוה עמל,"אוניברסיטת ת""א",Sunday,19,19,2,27,3
773,32.187855,34.877079,32.112881,34.818039,1,הדר-יוסף,Shazar,Tuesday,11,11,1,11,5


In [58]:
##### from CHAID import Tree, NominalColumn, ContinuousColumn
import pandas as pd
import numpy as np

# Assuming df_for_decision_tree is already defined
# Example DataFrame creation (remove these lines if df_for_decision_tree is provided elsewhere)
# ndarr = np.array(([1, 2, 3] * 5) + ([2, 2, 3] * 5)).reshape(10, 3)
# df = pd.DataFrame(ndarr)
# df.columns = ['a', 'b', 'c']
# arr = np.array(([1] * 5) + ([2] * 5))
# df['d'] = arr
# df_for_decision_tree = df

# Set the CHAID input parameters
independent_variable_columns = list(df_for_decision_tree.columns)
independent_variable_columns.remove('START_LATITUDE')
independent_variable_columns.remove('START_LONGITUDE')
independent_variable_columns.remove('END_LATITUDE')
independent_variable_columns.remove('END_LONGITUDE')
independent_variable_columns.remove('label')
independent_variable_columns.remove('end_NEIGHBORHOOD')
independent_variable_columns.remove('start_NEIGHBORHOOD')
independent_variable_columns.remove('DAY_OF_WEEK')
dep_variable = 'label'

# Define types for the columns
type_ = ['nominal'] * 4  # Adjust according to your dataset
type_ += ['nominal'] * 5  # Adjust according to your dataset

# Configuration dictionary
config = {
    'alpha_merge': 0.1,
    'max_depth': 40,
    'min_parent_node_size': 10,
    'min_child_node_size': 10,
    'max_splits': 6,
    'split_threshold': 0,
    'is_exhaustive': True
}

# Create the Tree using from_pandas_df
tree = Tree.from_pandas_df(
    df_for_decision_tree,
    dict(zip(independent_variable_columns, type_)),
    dep_variable,
    **config
)

# Build the tree (if not already built)
tree.build_tree()

# Print the tree structure
tree.print_tree()

# Get a TreeLib object
tree_lib = tree.to_tree()

# Access different nodes of the tree
first_node = tree.tree_store[0]
print("First Node:", first_node)

# Access properties of the node
print("First Node Members:", first_node.members)

# Access properties of the split
print("First Node Split p-value:", first_node.split.p)
print("First Node Split Score:", first_node.split.score)


b"([], {1: 217.0, 2: 203.0, 3: 123.0, 4: 70.0, 5: 82.0, 6: 80.0}, (end_NEIGHBORHOOD_encoded, p=4.206582238223043e-72, score=343.58589384259415, groups=[[1, 10, 45, 26, 27, 16, 29, 35, 2, 3, 8, 9, 33, 40, 4, 7, 12, 31, 34, 36, 42, 43, 11], [6, 13, 44, 23, 15, 28, 32, 17, 21, 37, 39, 24]]), dof=5))\n|-- ([1, 10, 45, 26, 27, 16, 29, 35, 2, 3, 8, 9, 33, 40, 4, 7, 12, 31, 34, 36, 42, 43, 11], {1: 215.0, 2: 202.0, 3: 55.0, 4: 21.0, 5: 34.0, 6: 31.0}, (start_NEIGHBORHOOD_encoded, p=3.248724305697242e-117, score=552.7551700626454, groups=[[1, 12, 29, 31, 46, 11, 21, 28, 27, 2, 4, 9, 10, 14, 16, 18, 19, 22, 26, 30, 35, 41, 45], [15, 17, 23, 24, 39]]), dof=5))\n|   |-- ([1, 12, 29, 31, 46, 11, 21, 28, 27, 2, 4, 9, 10, 14, 16, 18, 19, 22, 26, 30, 35, 41, 45], {1: 215.0, 2: 201.0, 3: 0, 4: 0, 5: 0, 6: 0}, (start_NEIGHBORHOOD_encoded, p=4.2110126071695765e-37, score=161.96476574906063, groups=[[1, 12, 29, 31, 46, 11, 21, 28, 27], [2, 4, 9, 10, 14, 16, 18, 19, 22, 26, 30, 35, 41, 45]]), dof=1))\n|  

In [59]:
# Print the accuracy
print("Accuracy of the CHAID Tree:")
print(tree.accuracy())

# Print the classification rules
print("\nClassification Rules:")
rules = tree.classification_rules()
for rule in rules:
    print(f"Node ID: {rule['node']}")
    for condition in rule['rules']:
        print(f"  Variable: {condition['variable']}, Data: {condition['data']} \n")

Accuracy of the CHAID Tree:
0.9483870967741935

Classification Rules:
Node ID: 4
  Variable: end_NEIGHBORHOOD_encoded, Data: [1, 7, 10, 11, 12, 16, 27, 29, 31, 34, 35, 36, 42, 43, 45] 

  Variable: start_NEIGHBORHOOD_encoded, Data: [1, 12, 29, 31, 46, 11, 21, 28, 27] 

  Variable: start_NEIGHBORHOOD_encoded, Data: [1, 12, 29, 31, 46, 11, 21, 28, 27, 2, 4, 9, 10, 14, 16, 18, 19, 22, 26, 30, 35, 41, 45] 

  Variable: end_NEIGHBORHOOD_encoded, Data: [1, 10, 45, 26, 27, 16, 29, 35, 2, 3, 8, 9, 33, 40, 4, 7, 12, 31, 34, 36, 42, 43, 11] 

Node ID: 7
  Variable: END_HOUR, Data: ['10', '17', '18', '8'] 

  Variable: start_NEIGHBORHOOD_encoded, Data: [11] 

  Variable: end_NEIGHBORHOOD_encoded, Data: [2, 4] 

  Variable: start_NEIGHBORHOOD_encoded, Data: [1, 12, 29, 31, 46, 11, 21, 28, 27] 

  Variable: start_NEIGHBORHOOD_encoded, Data: [1, 12, 29, 31, 46, 11, 21, 28, 27, 2, 4, 9, 10, 14, 16, 18, 19, 22, 26, 30, 35, 41, 45] 

  Variable: end_NEIGHBORHOOD_encoded, Data: [1, 10, 45, 26, 27, 16, 2

In [60]:
with open("/bigdata/users-home/dor/transpotation research/agg_data/chaid_tree.pickle", 'wb') as file:
        pickle.dump(tree, file)

In [52]:
%pip install graphviz pydot


Note: you may need to restart the kernel to use updated packages.


In [64]:
%pip install git+https://github.com/plotly/orca.git



  Cloning https://github.com/plotly/orca.git to /tmp/pip-req-build-7t20js1j
  Running command git clone --quiet https://github.com/plotly/orca.git /tmp/pip-req-build-7t20js1j
  Resolved https://github.com/plotly/orca.git to commit ab50a7d93d54241694e642d9e668d2dafab41607
  Preparing metadata (setup.py) ... done
  Created wheel for UNKNOWN: filename=UNKNOWN-1.3.1-py3-none-any.whl size=1781 sha256=2c11d0ef73069d854a19549a8f4766c72c54a8bab7ddf0eebd4ed216873472e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-e66cksy_/wheels/ad/2a/f6/2979e110e74161da1c131097a799a2365a1f91afbaa37956db
Successfully built UNKNOWN
Note: you may need to restart the kernel to use updated packages.


In [20]:
# with open(f"/bigdata/users-home/dor/transpotation research/agg_data/grouped_neighbors.pickle", 'wb') as handle:
#     pickle.dump(neigh_dict,handle)


## creating fixed drives clustrings with no  return drive (based on END_LONGITUDE)

In [39]:
dict_drives_n = {}
dict_sum_n ={}
dict_cor_n = {}
agg_drives2 = agg_drives_filt.copy()
i=1
while len(agg_drives2)>1:
    tag_drives_group,agg_drives2 = tag_drives(agg_drives2)
    
    driver1_lst  = get_drives(df, tag_drives_group,with_returns=False)
    driver1_df = cleaned[cleaned.index.isin(driver1_lst)][columns_og[:15]+['END_LATITUDE','END_LONGITUDE']]
    sum_drives = len(driver1_df)
    if sum_drives > 0:
        dict_cor_n[i] = (float(tag_drives_group.head(1)['END_LATITUDE']), float(tag_drives_group.head(1)['END_LONGITUDE']))

        dict_sum_n[i] = sum_drives
        dict_drives_n[i] = driver1_df
    i+=1
dict_sum_n = {k: v for k, v in sorted(dict_sum_n.items(), key=lambda item: item[1], reverse=True)}

dict_cor_n = {k: dict_cor_n[k] for k in dict_sum_n.keys()}
dict_drives_n = {k: dict_drives_n[k] for k in dict_sum_n.keys()}

print(dict_sum_n)


{1: 140, 2: 117, 3: 68, 6: 47, 4: 46, 5: 45, 9: 31, 8: 25, 10: 19, 7: 17, 11: 9}


In [40]:
keys_to_delete = [key for key, value in dict_sum_n.items() if value < 15]
for key in keys_to_delete:
    del dict_sum_n[key]
    del dict_drives_n[key]
    del dict_cor_n[key]

print(dict_sum_n)

{1: 140, 2: 117, 3: 68, 6: 47, 4: 46, 5: 45, 9: 31, 8: 25, 10: 19, 7: 17}


In [41]:
import folium

def create_map(dataframes):
    # Create a base map
    m = folium.Map(location=[0, 0], zoom_start=2)

    # Define colors for markers
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'gray', 'black', 'pink', 'yellow', 'cyan', 'magenta', 'teal', 'maroon', 'navy', 'olive', 'lavender', 'peach', 'turquoise', 'silver', 'gold']

    j= 0 
    # Iterate through the list of data frames and add markers to the map
    for  df in dataframes:
        color = colors[j % len(colors)]
        j+=1# Select color based on index
        for i, row in df.iterrows():
            lat, lon = scaled_df.loc[i]['END_LATITUDE'], scaled_df.loc[i]['END_LONGITUDE']
            folium.CircleMarker(location=[lat, lon], radius=5, color=color, fill=True, fill_color=color).add_to(m)
    return m
create_map([dict_drives_n[key] for key in dict_drives_n ])


In [42]:
x = pd.concat([dict_drives[group] for group in dict_drives],axis=0)
x = pd.merge(x, cleaned[[ 'START_HOUR']], on='DRIVE_ID', how='left')


In [26]:
hour_drive_dummies = pd.get_dummies(x['START_HOUR'], prefix='START_HOUR')

# Concatenate the one-hot encoded column with the original DataFrame x
x = pd.concat([x, hour_drive_dummies], axis=1)

In [27]:
x.drop(['START_HOUR','END_LATITUDE','END_LONGITUDE'],axis=1,inplace=True)

In [28]:
x.reset_index(drop=True,inplace=True)

In [43]:
from sklearn.cluster import DBSCAN
from sklearn import cluster, datasets, mixture
gmm = mixture.GaussianMixture(
        n_components=2,
        covariance_type="full",
       
    )

clustering =gmm.fit_predict(x)



In [44]:
import folium


m = folium.Map(location=[0, 0], zoom_start=2)

# Define colors for markers
colors = ['red', 'blue', 'green', 'orange', 'purple', 'gray', 'black', 'pink', 'yellow', 'cyan', 'magenta', 'teal', 'maroon', 'navy', 'olive', 'lavender', 'peach', 'turquoise', 'silver', 'gold']
colors = {2:'green',0:'blue',1:'red',3:'orange',4:'pink',5:'teal',6:'maroon',7:'lavender'}
j= 0
# Iterate through the list of data frames and add markers to the map
for  group in dict_cor:
    
    # Select color based on index
    for i, row in dict_drives[group].iterrows():
        color = colors[clustering[j]]
        lat, lon = scaled_df.loc[i]['END_LATITUDE'], scaled_df.loc[i]['END_LONGITUDE']
        folium.CircleMarker(location=[lat, lon], radius=5, color=color, fill=True, fill_color=color).add_to(m)
        j+=1
m

In [59]:

# Perform PCA to reduce dimensionality to three components
pca = PCA(n_components=3)
pca_components = pca.fit_transform(x)

# Adjust DBSCAN parameters
eps = 0.5  # Maximum distance between two samples
min_samples = 20  # Minimum number of samples in a neighborhood

# Perform DBSCAN clustering with limited clusters
clustering = DBSCAN(eps=eps, min_samples=min_samples).fit_predict(pca_components)


In [60]:
clustering

array([ 2,  0,  1,  2, -1, -1, -1, -1, -1, -1,  4, -1,  0, -1, -1, -1, 10,
        5,  3, -1,  4,  3, -1, -1,  0, -1,  2, -1, -1, -1,  0, -1,  0, -1,
       -1,  0, -1,  0,  3,  0,  3,  0,  5, -1,  8, -1,  4, -1,  0,  1,  0,
        3,  8, -1,  0,  5,  2,  3, -1,  6,  0, -1,  4, -1, -1, -1,  0,  0,
        0, -1,  0,  1,  2, -1, -1,  6,  2,  2,  7, -1, -1, -1, -1,  3,  3,
       -1,  0,  1,  7, -1,  0,  6,  0,  6,  6, -1, -1,  6,  1,  6, -1,  1,
       -1, -1,  6, -1, 10,  3,  5,  0,  6,  3,  0,  8,  3, -1, -1, -1,  7,
       -1,  6, -1,  0,  6,  1, -1,  0,  1, -1,  0, 10,  8, 10, -1,  0, -1,
       -1, -1,  4, -1, -1, -1,  1, -1,  0, -1, -1, -1,  6,  8, -1,  8,  4,
       -1, -1, 10, -1, -1, -1,  2, -1, -1,  5, -1, -1,  8,  1, -1,  3, -1,
        8,  4,  6, -1,  9, -1, -1,  1, -1, -1, -1, -1, -1,  5, -1, -1, -1,
       -1, -1,  4, -1,  8,  9, -1, -1,  6, -1, -1,  8, -1, -1, -1, -1,  8,
       -1,  3,  0,  1, -1, -1, -1, -1,  8,  4,  6, -1, -1, -1, -1, -1, -1,
        8,  1, -1, -1, -1

In [61]:
import folium


m = folium.Map(location=[0, 0], zoom_start=2)

# Define colors for markers
colors = ['red', 'blue', 'green', 'orange', 'purple', 'gray', 'black', 'pink', 'yellow', 'cyan', 'magenta', 'teal', 'maroon', 'navy', 'olive', 'lavender', 'peach', 'turquoise', 'silver', 'gold']
colors = {2:'green',0:'blue',1:'red',3:'orange',4:'pink',5:'teal',6:'maroon',7:'lavender',-1:'cyan',8:'magenta',9:'teal',10:'peach'}
j= 0
# Iterate through the list of data frames and add markers to the map
for  group in dict_cor:
    
    # Select color based on index
    for i, row in dict_drives[group].iterrows():
        color = colors[clustering[j]]
        lat, lon = scaled_df.loc[i]['END_LATITUDE'], scaled_df.loc[i]['END_LONGITUDE']
        folium.CircleMarker(location=[lat, lon], radius=5, color=color, fill=True, fill_color=color).add_to(m)
        j+=1
m

In [137]:
x = pd.concat([dict_drives[group] for group in dict_drives],axis=0)
x = pd.merge(x, cleaned[[ 'START_HOUR']], on='DRIVE_ID', how='left')

In [91]:
drives_hours = {1:[7,8,9,10,19,20,21,22,23],2:[11,12,13,14,15,16,17,18],3:[0,1,2,3,4,5,6]}

In [106]:
def find_hour_drive(start_hour, drives_hours=drives_hours):
    for key, hours_list in drives_hours.items():
        if start_hour in hours_list:
            return key
    return None  # Return None if start hour not found in any list

# Apply the function to create the new column
x['hour_drives'] = x['START_HOUR'].apply(find_hour_drive)

In [107]:
# x.drop(['START_HOUR'],axis=1,inplace=True)

In [138]:
x.drop(['END_LATITUDE','END_LONGITUDE'],axis=1,inplace=True)

In [139]:
x.reset_index(drop=True,inplace=True)

In [115]:
def f(combination, x):
    y = pd.Series([combination[j-1] for j in dict_drives for k in range(len(dict_drives[j]))])
    cluster_index_dict = {}
    for i in range(len(y)):
        cluster_index_dict[y[i]] = cluster_index_dict.get(y[i], [])
        cluster_index_dict[y[i]].append(i)

    cluster_means = [x.loc[cluster_index_dict[i]].groupby(['START_HOUR']).mean() for i in set(y)]
    
    within_cluster_variance = 0

    for cluster_mean, indices in zip(cluster_means, cluster_index_dict.values()):
        for index in indices:
            row = x.iloc[index]
            try:
                # Exclude 'START_HOUR' from the dropped columns
                within_cluster_variance += np.sum((row.drop('START_HOUR') - cluster_mean.loc[row['START_HOUR']])**2)
            except KeyError:
                # If 'START_HOUR' is not found in cluster_mean, default to the first element
                within_cluster_variance += np.sum((row.drop('START_HOUR') - cluster_mean.iloc[0])**2)
    return within_cluster_variance

In [140]:
import itertools

def get_combinations(values, n):
    combinations = []
    for combination in itertools.product(values, repeat=n):
        unique_values = set(combination)
        if len(unique_values) >= 2:
            combinations.append(combination)
    return combinations
    
# Example usage:
values = [1,2]
n = len(dict_drives)
combinations = get_combinations(values, n)
print(len(combinations))
len(dict_drives)

62


6

In [141]:
max_var = 1000000000
max_comb = []

for combination in combinations:
    
    curr_var = f(combination,x)
    if curr_var < max_var:
        max_var = curr_var
        max_comb = combination

In [72]:
f(max_comb,x)

7675.284607887037

In [73]:
max_comb

(1, 2, 2, 1, 1, 1, 2, 1, 1, 1)

In [142]:
import folium


m = folium.Map(location=[0, 0], zoom_start=2)

# Define colors for markers
colors = ['red', 'blue', 'green', 'orange', 'purple', 'gray', 'black', 'pink', 'yellow', 'cyan', 'magenta', 'teal', 'maroon', 'navy', 'olive', 'lavender', 'peach', 'turquoise', 'silver', 'gold']
colors = {2:'green',0:'blue',1:'red',3:'orange',4:'pink',5:'teal',6:'maroon',7:'lavender'}
j= 0
# Iterate through the list of data frames and add markers to the map
for  group in dict_cor:
    color = colors[max_comb[group-1]]
    j+=1
    # Select color based on index
    if group in dict_drives_n:
        for i, row in dict_drives_n[group].iterrows():

            lat, lon = scaled_df.loc[i]['END_LATITUDE'], scaled_df.loc[i]['END_LONGITUDE']
            folium.CircleMarker(location=[lat, lon], radius=5, color=color, fill=True, fill_color=color).add_to(m)
m

In [143]:
for group in dict_cor:
    neighborhood_name = get_neighborhood_name(dict_cor[group][0], dict_cor[group][1])
    print("Neighborhood:", translate_hebrew_to_english(neighborhood_name),'| Number of drives:', len(dict_drives[group]))
    print('---------------------------------------------------------------\n')

Neighborhood: Hadar-Yosef | Number of drives: 243
---------------------------------------------------------------

Neighborhood: Tel Aviv University | Number of drives: 228
---------------------------------------------------------------

Neighborhood: Neot Uzi | Number of drives: 124
---------------------------------------------------------------

Neighborhood: Neot afeka a. | Number of drives: 83
---------------------------------------------------------------

Neighborhood: Rom 2000 | Number of drives: 80
---------------------------------------------------------------

Neighborhood: Herzliya Pituach | Number of drives: 70
---------------------------------------------------------------



In [ ]:
irads = ['Tel Aviv University','Herzliya Pituach','Hof Hasharon Regional Council']
hilis = ['Hadar-Yosef','Rom 2000','Neot afeka a.']

In [104]:
for group in dict_drives:
    lst = []
    curr_df = pd.merge(dict_drives[group], cleaned[[ 'START_HOUR']], on='DRIVE_ID', how='left')
    curr_df['hour_drives'] = curr_df['START_HOUR'].apply(find_hour_drive)
    
    
    
#     hour_drive_dummies = pd.get_dummies(curr_df['hour_drives'], prefix='hour_drive')

#     # Concatenate the one-hot encoded column with the original DataFrame x
#     curr_df = pd.concat([curr_df, hour_drive_dummies], axis=1)
    curr_df.drop(['START_HOUR'],axis=1,inplace=True)
    # Calculate mean for each hour_drive
    mean_df = curr_df.groupby('hour_drives').mean()

    # Calculate standard deviation for each hour_drive
    std_df = curr_df.groupby('hour_drives').std()

    # Find rows with values more than 3.5 std from mean
    for index, row in curr_df.iterrows():
        if any(row[col] > mean_df.loc[row['hour_drives'], col] + 3.5 * std_df.loc[row['hour_drives'], col] 
               for col in mean_df.columns):
            lst.append(row)
            dict_drives[group] = dict_drives[group][dict_drives[group].index != index]
    print('original df',len(dict_drives[group]))
    print('outliars rows',len(lst))

original df 213
outliars rows 16
original df 195
outliars rows 19
original df 107
outliars rows 3
original df 78
outliars rows 5
original df 76
outliars rows 4
original df 61
outliars rows 7


In [144]:
import pickle

with open("/bigdata/users-home/dor/transpotation research/agg_data/preprocessGlobalFeatures/2_2019_[460631]_trips.pickle", 'rb') as handle:
    trips_lst = pickle.load(handle)

In [145]:
print(len(trips_lst))
for i in trips_lst[0]:
    print(i.time_series_record)

1
    vehicle_id  longitude   latitude           orig_time  speed  direction  \
11      460631  34.857728  32.165491 2019-02-01 06:40:17      0          0   
13      460631  34.857728  32.165491 2019-02-01 06:40:18      0          0   
14      460631  34.857728  32.165500 2019-02-01 06:40:21      0          0   
15      460631  34.857723  32.165503 2019-02-01 06:40:25      0          0   
16      460631  34.857713  32.165500 2019-02-01 06:40:29      0          0   
..         ...        ...        ...                 ...    ...        ...   
79      460631  34.843315  32.160441 2019-02-01 06:49:55      0          0   
80      460631  34.843321  32.160440 2019-02-01 06:50:05      0          0   
81      460631  34.843335  32.160446 2019-02-01 06:50:15      0          0   
82      460631  34.843345  32.160458 2019-02-01 06:50:22      0          0   
83      460631  34.843348  32.160468 2019-02-01 06:50:27      0          0   

    vehicle_state       mileage  road_speed  
11             

       vehicle_id  longitude   latitude           orig_time  speed  direction  \
10017      460631  34.809688  32.160395 2019-02-11 16:58:32      7        120   
10018      460631  34.809688  32.160395 2019-02-11 16:58:32      8        120   
10019      460631  34.809688  32.160395 2019-02-11 16:58:35      8        120   
10020      460631  34.809688  32.160395 2019-02-11 16:58:39      4        120   
10021      460631  34.809688  32.160395 2019-02-11 16:58:43      2        120   
...           ...        ...        ...                 ...    ...        ...   
10242      460631  34.828941  32.221233 2019-02-11 17:35:54      0          0   
10243      460631  34.828993  32.221228 2019-02-11 17:36:04      0          0   
10244      460631  34.829015  32.221230 2019-02-11 17:36:08      0          0   
10245      460631  34.829015  32.221230 2019-02-11 17:36:09      0          0   
10246      460631  34.829025  32.221226 2019-02-11 17:36:13      0          0   

       vehicle_state       

In [154]:
import folium

# Create a map object
m = folium.Map(location=[0, 0], zoom_start=2)

# Define colors for markers
colors = {2: 'green', 0: 'blue', 1: 'red', 3: 'orange', 4: 'pink', 5: 'teal', 6: 'maroon', 7: 'lavender'}

color = colors[0]
j = 0 

# Select color based on index
for trip in trips_lst[0]:
    color = colors[j]
    j += 1
    if j < 5:
        for i, row in trip.time_series_record.iterrows():
            
            lat, lon = row['latitude'], row['longitude']
            folium.CircleMarker(location=[lat, lon], radius=5, color=color, fill=True, fill_color=color).add_to(m)
    else:
        break
# Display the map
m
